In [1]:
!pip install -Uq nest_asyncio langchain openai lxml bs4

zsh:1: /Users/yjkim/Developer/WhatsUpDoc/rag-notebooks/venv/bin/pip: bad interpreter: /Users/yjkim/Developer/whatsupdocc/venv/bin/python3.11: no such file or directory

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
from langchain.document_loaders.sitemap import SitemapLoader
# fixes a bug with asyncio and jupyter
import nest_asyncio

nest_asyncio.apply()

In [13]:
# sitemap_loader = SitemapLoader(
#     web_path="https://python.langchain.com/sitemap.xml",
#     )

# should skip https://js.langchain.com/docs/modules/agents/tools/dynamic
sitemap_loader = SitemapLoader(
    web_path="./sitemap.xml",
    is_local=True
)

sitemap_loader.requests_per_second = 2

# Optional: avoid `[SSL: CERTIFICATE_VERIFY_FAILED]` issue
# sitemap_loader.requests_kwargs = {"verify": False}

# docs = sitemap_loader.load()

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

data = sitemap_loader.load_and_split(text_splitter=text_splitter)

docs = text_splitter.split_documents(data)

Fetching pages:  64%|######4   | 307/478 [08:19<04:38,  1.63s/it]


In [16]:
!pip install -Uq supabase tiktoken python-dotenv

zsh:1: /Users/yjkim/Developer/WhatsUpDoc/rag-notebooks/venv/bin/pip: bad interpreter: /Users/yjkim/Developer/whatsupdocc/venv/bin/python3.11: no such file or directory

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [17]:
from supabase.client import Client, create_client
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.supabase import SupabaseVectorStore
import os
from dotenv import load_dotenv

load_dotenv()

supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_SERVICE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

embeddings = OpenAIEmbeddings()

In [18]:
len(docs)

7637

In [25]:
vector_store = SupabaseVectorStore.from_documents(
    documents=docs[1000:],
    embedding=embeddings,
    client=supabase,
    table_name="langchainjs_documents",
    query_name="langchainjs_match_documents",
    chunk_size=100
)

2024-02-04 13:32:20,346:INFO - HTTP Request: POST https://qublpyarwoevdeqqicbz.supabase.co/rest/v1/langchainjs_documents "HTTP/1.1 201 Created"
2024-02-04 13:32:25,642:INFO - HTTP Request: POST https://qublpyarwoevdeqqicbz.supabase.co/rest/v1/langchainjs_documents "HTTP/1.1 201 Created"
2024-02-04 13:32:30,846:INFO - HTTP Request: POST https://qublpyarwoevdeqqicbz.supabase.co/rest/v1/langchainjs_documents "HTTP/1.1 201 Created"
2024-02-04 13:32:36,904:INFO - HTTP Request: POST https://qublpyarwoevdeqqicbz.supabase.co/rest/v1/langchainjs_documents "HTTP/1.1 201 Created"
2024-02-04 13:32:42,037:INFO - HTTP Request: POST https://qublpyarwoevdeqqicbz.supabase.co/rest/v1/langchainjs_documents "HTTP/1.1 201 Created"
2024-02-04 13:32:46,292:INFO - HTTP Request: POST https://qublpyarwoevdeqqicbz.supabase.co/rest/v1/langchainjs_documents "HTTP/1.1 201 Created"
2024-02-04 13:32:51,524:INFO - HTTP Request: POST https://qublpyarwoevdeqqicbz.supabase.co/rest/v1/langchainjs_documents "HTTP/1.1 201 C

In [26]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.retrievers import RePhraseQueryRetriever
from langchain.memory import ConversationBufferMemory
from langchain.chains import create_citation_fuzzy_match_chain


memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

DEFAULT_TEMPLATE = """You are an assistant tasked with taking a natural language \
query from a user and converting it into a query for a vectorstore. \
In this process, you strip out information that is not relevant for \
the retrieval task. Here is the user query: {question}"""

llm = ChatOpenAI(temperature=0)
retriever_from_llm = RePhraseQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(), llm=llm
)
# docs = retriever_from_llm.get_relevant_documents("How do I load documents from Hacker News?")

# qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectorstore.as_retriever(), memory=memory)

# chain = create_citation_fuzzy_match_chain(llm)

qa = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever_from_llm, memory=memory)


In [27]:
qa({ "question": "How do I get a Retrieval chain to cite its sources?"})

2024-02-04 13:38:07,398:INFO - Re-phrased question: To convert the user query into a query for a vectorstore, you can strip out the irrelevant information and focus on the essential elements. The modified query would be:

"Retrieval chain cite sources"

This query captures the user's intention of wanting to know how to make a retrieval chain cite its sources.
2024-02-04 13:38:09,193:INFO - HTTP Request: POST https://qublpyarwoevdeqqicbz.supabase.co/rest/v1/rpc/langchainjs_match_documents?limit=4 "HTTP/1.1 404 Not Found"


APIError: {'code': 'PGRST202', 'details': 'Searched for the function public.langchainjs_match_documents with parameter query_embedding or with a single unnamed json/jsonb parameter, but no matches were found in the schema cache.', 'hint': 'Perhaps you meant to call the function public.match_langchainjs_documents', 'message': 'Could not find the function public.langchainjs_match_documents(query_embedding) in the schema cache'}